<a href="https://colab.research.google.com/github/treasure-data/td-notebooks/blob/master/machine-learning/a9a/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This notebook is a tutrial for Hivemall, based on [here](http://hivemall.incubator.apache.org/userguide/binaryclass/a9a_generic.html).  
In addition, it shows a comparison of optimizing methods.

## SetUp

Executing a following cell, you are asked to input some information.

In [1]:
from getpass import getpass
td_api_key = getpass('Enter TD API KEY here ')
database   = input('Enter YOUR DATABASE to create tables here ')
assert database, "Error: Input YOUR DATABASE"
endpoint   = input('Enter YOUR ENDPOINT to create tables here (default: "api.treasuredata.com") \nref : https://support.treasuredata.com/hc/en-us/articles/360001474288-Sites-and-Endpoints ')
endpoint = endpoint if endpoint else 'https://api.treasuredata.com/'
endpoint = 'https://' + endpoint if not endpoint.startswith('https://') else endpoint
use_spark  = input('If enable to use spark, enter "true" (default: false) ').lower()
use_spark  = use_spark == 'true'
table = 'churn_dataset'

Enter TD API KEY here ··········
Enter YOUR DATABASE to create tables here hivemall
Enter YOUR ENDPOINT to create tables here (default: "api.treasuredata.com") 
ref : https://support.treasuredata.com/hc/en-us/articles/360001474288-Sites-and-Endpoints 
If enable to use spark, enter "true" (default: false) true


In [0]:
%%capture
!pip install td-client
!pip install pandas-td
!pip install -q git+https://github.com/takuti/pytd@master#egg=treasure-data[spark]

import numpy as np
import pandas as pd
import pandas_td as td
import matplotlib.pyplot as plt
import seaborn as sns
import pytd
import tdclient
pd.options.display.max_columns = 30

%matplotlib inline
con = td.connect(apikey=td_api_key, endpoint=endpoint)
c = tdclient.Client(apikey=td_api_key, endpoint=endpoint)
presto = td.create_engine('presto:{}'.format(database), con=con)
hive = td.create_engine('hive:{}'.format(database), con=con)

def recreate_table(client, table, database=database):
  if table in [table.name for table in client.tables(database)]:
    client.delete_table(database, table)
  client.create_log_table(database, table)
  return None

## Data Prepraration

In [0]:
%%capture
# Download 
!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a
!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a.t
!wget https://raw.githubusercontent.com/apache/incubator-hivemall/master/resources/misc/conv.awk

In [0]:
# Preprocess
!awk -f conv.awk a9a | sed -e "s/+1/1/" | sed -e "s/-1/0/" > a9a.train
!awk -f conv.awk a9a.t | sed -e "s/+1/1/" | sed -e "s/-1/0/" > a9a.test

train = pd.read_csv('./a9a.train', sep='\t')
train.columns = ['id','target','features']
test = pd.read_csv('./a9a.test', sep='\t')
test.columns = ['id','target','features']

In [0]:
# Upload
if use_spark:
  client = pytd.Client(database=database, apikey=td_api_key)
  client.load_table_from_dataframe(train, '{}.{}'.format(database, 'a9a_train'), if_exists='overwrite')
  client.load_table_from_dataframe(test, '{}.{}'.format(database, 'a9a_test'), if_exists='overwrite')
else:
  for table in ['a9a_train', 'a9a_test']:
    if table in [table.name for table in c.tables(database)]:
      c.delete_table(database, table)
  td.to_td(train, '{}.{}'.format(database, 'a9a_train'), con, index=False)
  td.to_td(test,  '{}.{}'.format(database, 'a9a_test'), con, index=False)
  import time
  time.sleep(300)

## EDA (Explore Data Analysis)
Before running machine learning tasks, you should understand your data correctly.  
Descriptive statistics and visualization are important for this.

### Sample

In [6]:
td.read_td_query('''
select
  *
from
  a9a_train
limit 2
''', engine=presto)

,id,target,features,time
0,2,0,"5:1,7:1,14:1,19:1,39:1,40:1,51:1,63:1,67:1,73:...",1550827306
1,3,0,"3:1,6:1,17:1,22:1,36:1,41:1,53:1,64:1,67:1,73:...",1550827306


In [7]:
td.read_td_query('''
select
  *
from
  a9a_test
limit 2
''', engine=presto)

,id,target,features,time
0,2,0,"3:1,6:1,14:1,22:1,36:1,40:1,56:1,63:1,67:1,73:...",1550827316
1,3,1,"2:1,10:1,18:1,24:1,38:1,40:1,59:1,63:1,67:1,73...",1550827316


### Data Size

In [8]:
td.read_td_query('''
select 'train', count(*) as n from a9a_train
union all
select 'test', count(*) as n from a9a_test
''', engine=presto)

,_col0,n
0,train,32560
1,test,16280


### The number of features

In [9]:
td.read_td_query('''
select
  'train'
  ,count(distinct split(feature, ':')[1]) as n
  ,min(cast(split(feature, ':')[1] as int)) as min
  ,max(cast(split(feature, ':')[1] as int)) as max
from
  a9a_train
CROSS JOIN UNNEST(split(features,',')) AS t (feature)
union all
select
  'test'
  ,count(distinct split(feature, ':')[1]) as n
  ,min(cast(split(feature, ':')[1] as int)) as min
  ,max(cast(split(feature, ':')[1] as int)) as max
from
  a9a_test
CROSS JOIN UNNEST(split(features,',')) AS t (feature)
''', engine=presto)

,_col0,n,min,max
0,test,122,1,122
1,train,123,1,123


There is a column that training data has but test data doesn't have.

### The number of samples for each classes

In [10]:
td.read_td_query('''
select
  target
  ,count(*) as n
from
  a9a_train
group by
  1
order by
  1
''', engine=presto)

,target,n
0,0,24719
1,1,7841


In [11]:
td.read_td_query('''
select
  target
  ,count(*) as n
from
  a9a_test
group by
  1
order by
  1
''', engine=presto)

,target,n
0,0,12434
1,1,3846


## Build models

In [0]:
recreate_table(c, 'a9a_model')

In [13]:
%%time
opts = ['sgd', 'Momentum', 'Nesterov', 'AdaGrad', 'RMSprop', 'RMSpropGraves', 'AdaDelta', 'Adam', 'Nadam', 'Eve', 'AdamHD']
for opt in opts:
  _opt = ' -opt {}'.format(opt)
  td.read_td_query('''
    insert into table a9a_model 
    select 
      '{opt}' as opt,
      feature,
      avg(weight) as weight
    from (
      select 
         train_classifier(
           add_bias(split(features, ',')), target, 
           "-loss logistic -iter 30 {_opt} -reg l1"
         ) as (feature, weight)
      from 
         a9a_train
     ) t 
    group by 
      1,2
    '''.format(opt=opt, _opt=_opt), engine=hive)

CPU times: user 1.33 s, sys: 199 ms, total: 1.53 s
Wall time: 5min 21s


Check features with top-5 positive weights.

In [14]:
td.read_td_query('''
select
  *
from
  a9a_model
order by
  weight desc
limit 5
''', engine=presto)

,opt,feature,weight,time
0,RMSpropGraves,75,4.479211,1550827474
1,RMSpropGraves,0,3.613928,1550827474
2,Nesterov,51,2.574403,1550827391
3,Momentum,46,2.573963,1550827364
4,Momentum,51,2.573463,1550827364


Check features with top-5 negative weights.

In [15]:
td.read_td_query('''
select
  *
from
  a9a_model
order by
  weight
limit 5
''', engine=presto)

,opt,feature,weight,time
0,Eve,34,-4.938404,1550827592
1,Adam,34,-4.879388,1550827530
2,Nadam,34,-4.664753,1550827559
3,Nadam,12,-4.493797,1550827559
4,Eve,12,-4.136467,1550827592


## Predict

In [0]:
recreate_table(c, 'a9a_predict')

In [17]:
td.read_td_query('''
with t as (
  select
    'train' as class
    ,id
    ,target
    ,extract_feature(feature) as feature
    ,extract_weight(feature) as value
  from
    a9a_train
  lateral view
    explode(split(features, ',')) t as feature

  union all
    
  select
    'test' as class
    ,id
    ,target
    ,extract_feature(feature) as feature
    ,extract_weight(feature) as value
  from
    a9a_test
  lateral view
    explode(split(features, ',')) t as feature
)

insert into table a9a_predict
select
  id
  ,opt
  ,class
  ,target
  ,sigmoid(sum(value * weight)) as score
from
  t
  inner join
  a9a_model m
  on
    t.feature = m.feature
group by
  1,2,3,4
''', engine=hive)

,id,opt,class,target,score


In [18]:
td.read_td_query('''
select
  *
from
  a9a_predict
limit 10
''', engine=presto)

,id,opt,class,target,score,time
0,2,AdaDelta,test,0,0.171574,1550827665
1,2,AdaDelta,train,0,0.351619,1550827665
2,2,AdaGrad,test,0,0.202097,1550827665
3,2,AdaGrad,train,0,0.384879,1550827665
4,2,Adam,test,0,0.120721,1550827665
5,2,Adam,train,0,0.346950,1550827665
6,2,AdamHD,test,0,0.137380,1550827665
7,2,AdamHD,train,0,0.303265,1550827665
8,2,Eve,test,0,0.104889,1550827665
9,2,Eve,train,0,0.403007,1550827665


In [0]:
aucs = td.read_td_query('''
select 'train' as class, 'sgd'           as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'sgd'           and class = 'train' order by score desc) t union all
select 'test'  as class, 'sgd'           as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'sgd'           and class = 'test'  order by score desc) t union all
select 'train' as class, 'Momentum'      as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Momentum'      and class = 'train' order by score desc) t union all
select 'test'  as class, 'Momentum'      as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Momentum'      and class = 'test'  order by score desc) t union all
select 'train' as class, 'Nesterov'      as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Nesterov'      and class = 'train' order by score desc) t union all
select 'test'  as class, 'Nesterov'      as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Nesterov'      and class = 'test'  order by score desc) t union all
select 'train' as class, 'AdaGrad'       as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'AdaGrad'       and class = 'train' order by score desc) t union all
select 'test'  as class, 'AdaGrad'       as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'AdaGrad'       and class = 'test'  order by score desc) t union all
select 'train' as class, 'RMSprop'       as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'RMSprop'       and class = 'train' order by score desc) t union all
select 'test'  as class, 'RMSprop'       as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'RMSprop'       and class = 'test'  order by score desc) t union all
select 'train' as class, 'RMSpropGraves' as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'RMSpropGraves' and class = 'train' order by score desc) t union all
select 'test'  as class, 'RMSpropGraves' as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'RMSpropGraves' and class = 'test'  order by score desc) t union all
select 'train' as class, 'AdaDelta'      as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'AdaDelta'      and class = 'train' order by score desc) t union all
select 'test'  as class, 'AdaDelta'      as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'AdaDelta'      and class = 'test'  order by score desc) t union all
select 'train' as class, 'Adam'          as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Adam'          and class = 'train' order by score desc) t union all
select 'test'  as class, 'Adam'          as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Adam'          and class = 'test'  order by score desc) t union all
select 'train' as class, 'Nadam'         as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Nadam'         and class = 'train' order by score desc) t union all
select 'test'  as class, 'Nadam'         as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Nadam'         and class = 'test'  order by score desc) t union all
select 'train' as class, 'Eve'           as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Eve'           and class = 'train' order by score desc) t union all
select 'test'  as class, 'Eve'           as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'Eve'           and class = 'test'  order by score desc) t union all
select 'train' as class, 'AdamHD'        as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'AdamHD'        and class = 'train' order by score desc) t union all
select 'test'  as class, 'AdamHD'        as opt, auc(score, target) as auc from ( select id, target, score from a9a_predict where opt = 'AdamHD'        and class = 'test'  order by score desc) t
''', engine=hive)

In [21]:
aucs.pivot_table(index='opt', columns='class', values='auc')[['train', 'test']].sort_values('test', ascending=False)

class,train,test
opt,,
AdaGrad,0.903526,0.902283
AdaDelta,0.901916,0.900185
RMSprop,0.901343,0.898278
AdamHD,0.901814,0.898117
sgd,0.901149,0.898077
Nadam,0.894220,0.889895
Eve,0.894237,0.889884
Adam,0.894062,0.889763
Nesterov,0.881875,0.877429
